In [1]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice

c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\exploitation'

In [3]:
data_f=pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f=pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

In [4]:
data_f.head()

,user_id,sig_id,fakeness
0,100,1,True
1,100,2,True
2,100,3,True
3,100,4,True
4,100,5,True


In [5]:
visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,0.515184,0.0,0.446658,0.000000,1.533631,0.000000,0.830734,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.557626,2.271116,0.000000,0.000000,0.0
1,0.775465,0.491479,0.0,0.000000,0.000000,1.406079,0.000000,0.492579,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.852994,0.000000,0.114499,0.0
2,0.000000,0.000000,0.0,0.536446,0.000000,1.768807,0.000000,0.676527,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,2.392846,0.000000,0.226075,0.0
3,0.558339,0.000000,0.0,0.693711,0.166456,0.771811,0.643744,0.890198,1.242508,0.0,...,0.0,0.036022,0.000000,0.0,0.0,0.000000,1.391226,0.653054,0.000000,0.0
4,0.000000,0.000000,0.0,0.000000,0.000000,0.227453,1.312067,1.143076,1.002350,0.0,...,0.0,0.000000,0.067551,0.0,0.0,0.000000,0.892711,0.037287,0.196272,0.0


In [6]:
data_f_gen = data_f.loc[data_f['fakeness']==False]
data_f_fake = data_f.loc[data_f['fakeness']==True]
data_f_gen_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(1,13))]
data_f_gen_valid_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(13,25))]

In [19]:
scores_sum = 0
run_count = 0
for user_id in tqdm.tqdm(data_f['user_id'].unique(), ascii=True):
    y_train = data_f_gen_12['user_id']==user_id
    X_train = visual_f.loc[y_train.index]
    class_weight = (data_f_gen_12.shape[0]-data_f_gen_12[y_train].shape[0])/data_f_gen_12[y_train].shape[0]
    # clf = XGBClassifier(tree_method='gpu_hist',verbosity=2, scale_pos_weight=class_weight)
    clf = SVC(class_weight='balanced',gamma='scale')
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
    y_valid = len(y_valid_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_fakes.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']==user_id]
    y_valid = len(y_valid_gens.index)*[True]
    X_valid = visual_f.loc[y_valid_gens.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_rand_fakes = data_f_gen_valid_12.loc[choice(data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']!=user_id].index,10)]
    y_valid = len(y_valid_rand_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    # print(scores_sum/run_count)
print(scores_sum/run_count)


  0%|                                                                                          | 0/300 [00:00<?, ?it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

  0%|2                                                                                 | 1/300 [00:00<04:27,  1.12it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

  1%|5                                                                                 | 2/300 [00:01<03:57,  1.25it/s]c:\u


  6%|#####1                                                                           | 19/300 [00:12<03:07,  1.50it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

  7%|#####4                                                                           | 20/300 [00:13<03:23,  1.37it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

  7%|#####6                                                                           | 21/300 [00:14<03:16,  1.42it/s]c:\u


 13%|##########2                                                                      | 38/300 [00:26<03:03,  1.42it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 13%|##########5                                                                      | 39/300 [00:27<03:02,  1.43it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 13%|##########8                                                                      | 40/300 [00:28<03:01,  1.43it/s]c:\u


 19%|###############3                                                                 | 57/300 [00:40<03:17,  1.23it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 19%|###############6                                                                 | 58/300 [00:41<03:06,  1.29it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 20%|###############9                                                                 | 59/300 [00:41<03:04,  1.31it/s]c:\u


 25%|####################5                                                            | 76/300 [00:54<02:53,  1.29it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 26%|####################7                                                            | 77/300 [00:55<02:47,  1.33it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 26%|#####################                                                            | 78/300 [00:55<02:31,  1.46it/s]c:\u

...


 76%|############################################################8                   | 228/300 [02:46<00:53,  1.35it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 76%|#############################################################                   | 229/300 [02:47<00:50,  1.40it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 77%|#############################################################3                  | 230/300 [02:48<00:49,  1.40it/s]c:\u


 82%|#################################################################8              | 247/300 [03:01<00:34,  1.53it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 83%|##################################################################1             | 248/300 [03:01<00:34,  1.53it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 83%|##################################################################3             | 249/300 [03:02<00:34,  1.46it/s]c:\u


 89%|######################################################################9         | 266/300 [03:15<00:25,  1.32it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 89%|#######################################################################2        | 267/300 [03:16<00:24,  1.32it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 89%|#######################################################################4        | 268/300 [03:17<00:23,  1.34it/s]c:\u


 95%|############################################################################    | 285/300 [03:29<00:12,  1.23it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 95%|############################################################################2   | 286/300 [03:29<00:10,  1.33it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)

 96%|############################################################################5   | 287/300 [03:30<00:09,  1.38it/s]c:\u

0.9363253505115564


In [8]:
'''
y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
y_valid = len(y_valid_fakes.index)*[False]
X_valid = visual_f.loc[y_valid_fakes.index]
'''

"\ny_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]\ny_valid = len(y_valid_fakes.index)*[False]\nX_valid = visual_f.loc[y_valid_fakes.index]\n"

In [18]:
y_train.head()

8905    False
8906    False
8907    False
8908    False
8909    False
Name: user_id, dtype: bool